In [1]:
# Let's load and process the dataset
from fuel.datasets.dogs_vs_cats import DogsVsCats
from fuel.schemes import SequentialScheme
from fuel.server import start_server
from fuel.streams import DataStream
from fuel.transformers.image import RandomFixedSizeCrop, MinimumImageDimensions
from fuel.transformers import Flatten, ScaleAndShift, Cast
import numpy
import socket
import sys
from transformers import FixedSizeCrops, DownscaleMinDimension

batch_size = 25

# Load the training set
train = DogsVsCats(('test',))
# We now create a "stream" over the dataset which will return shuffled batches
# of size 128. Using the DataStream.default_stream constructor will turn our
# 8-bit images into floating-point decimals in [0, 1].
stream = DataStream.default_stream(
    train,
    iteration_scheme=SequentialScheme(train.num_examples, batch_size)
)

upscaled_stream = MinimumImageDimensions(stream, (100, 100), which_sources=('image_features',))
downscaled_stream = DownscaleMinDimension(upscaled_stream, 100, which_sources=('image_features',))

# Our images are of different sizes, so we'll use a Fuel transformer
# to take random crops of size (32 x 32) from each image
cropped_stream = FixedSizeCrops(
    downscaled_stream, (100, 100), which_sources=('image_features',))

# We'll use a simple MLP, so we need to flatten the images
# from (channel, width, height) to simply (features,)
float_stream = ScaleAndShift(cropped_stream, 1./255, 0, which_sources=('image_features',))
float32_stream = Cast(float_stream, numpy.float32, which_sources=('image_features',))


Using gpu device 0: GeForce GTX TITAN Black (CNMeM is disabled, CuDNN 4007)


In [2]:
from blocks.extensions.saveload import load

with open('/tmp/train_bn2', 'rb') as src:
    main_loop_loaded = load(src)

Using saved session configuration for http://localhost:5006
To override, pass 'load_from_config=False' to Session


In [3]:
model = main_loop_loaded.model

In [10]:
ei = float32_stream.get_epoch_iterator()
d = next(ei)

print d[0].shape
print len(d)

(25, 3, 100, 200)
1


In [5]:
import theano
from theano import tensor

X = tensor.tensor4('image_features', dtype='float32')
f = theano.function(model.inputs, model.outputs)


In [6]:
theano.config.exception_verbosity = 'low'

f(d[0])

[array([[  9.99719560e-01],
        [  9.99987602e-01],
        [  9.99820054e-01],
        [  9.99993920e-01],
        [  8.32583464e-04],
        [  7.41220079e-04],
        [  2.30501202e-04],
        [  2.69188226e-04],
        [  1.15784153e-03],
        [  1.15517469e-04],
        [  5.27473057e-05],
        [  9.99813616e-01],
        [  2.82882102e-04],
        [  2.68404779e-04],
        [  6.03929511e-04],
        [  8.00209679e-03],
        [  9.99966383e-01],
        [  9.99957800e-01],
        [  1.26400764e-05],
        [  1.13869646e-04],
        [  9.99950528e-01],
        [  1.14523026e-03],
        [  9.99976993e-01],
        [  9.78722751e-01],
        [  1.67087637e-04]], dtype=float32)]

In [21]:
main_loop_loaded.algorithm.step_rule.learning_rate.set_value(0.001)

In [ ]:
main_loop_loaded.run()


-------------------------------------------------------------------------------
TRAINING HAS BEEN RESUMED
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: True
	 epoch_started: False
	 epochs_done: 119
	 iterations_done: 71380
	 received_first_batch: True
	 resumed_from: 9cdd96d3a60f42e5ac24da2da7526f50
	 training_started: True
Log records from the iteration 71380:
	 error: 0.0347334221005
	 loss: 0.0833946987987
	 time_read_data_this_epoch: 0.376342535019
	 time_read_data_total: 45.5398755074
	 time_train_this_epoch: 453.025973797
	 time_train_total: 53913.5800719
	 training_finished: True
	 valid_error: 0.0489999689162



## Prepare submission

In [7]:
ei = float32_stream.get_epoch_iterator()
prediction = []

for d in ei:
    prediction.append(f(d[0])[0])

In [9]:
print len(prediction)*prediction[0].shape[0]

12500


In [13]:
out_csv = open('output.csv','w')
out_csv.write('id,label\n')
i=1
for p_batch in prediction:
    for p in p_batch:
        out_csv.write('%d,%d\n' % (i, (p>0.5)*1))
        i += 1
                      
out_csv.close()